## Overview

This notebook shows you how to create and query a table or DataFrame loaded from data stored in AWS S3. There are two ways to establish access to S3: [IAM roles](https://docs.databricks.com/user-guide/cloud-configurations/aws/iam-roles.html) and access keys.

*We recommend using IAM roles to specify which cluster can access which buckets. Keys can show up in logs and table metadata and are therefore fundamentally insecure.* If you do use keys, you'll have to escape the `/` in your keys with `%2F`.

This is a **Python** notebook so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` magic command. Python, Scala, SQL, and R are all supported.

In [2]:
# Access S3 bucket and mount it into Databricks. Check if a mount exists or not before creating a new mount. 
import urllib
ACCESS_KEY = "*****"
SECRET_KEY = "*****"
SECRET_KEY= SECRET_KEY.replace('/','%2F')
ENCODED_SECRET_KEY = urllib.parse.quote(SECRET_KEY, "")
AWS_BUCKET_NAME = "*****"
MOUNT_NAME = "Northwoods"
if any(mount.mountPoint == '/mnt/%s' for mount in dbutils.fs.mounts()):
  dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

display(dbutils.fs.ls("/mnt/Northwoods"))


path,name,size
dbfs:/mnt/Northwoods/airlines.csv,airlines.csv,359
dbfs:/mnt/Northwoods/airports.csv,airports.csv,23867
dbfs:/mnt/Northwoods/flights.csv,flights.csv,592406591


In [3]:
# 1. Get data from mounted files into dataframes. 
# 2. Push the dataframe to delta files (overwrite to allow repeated executions).
# 3. Push the dataframe to Snowflake (overwrite to allow repeated executions).


# Snowflake Credentials
sfOptions = dict(
  sfURL = "*****",
  sfUser = "*****",
  sfPassword = "*****",
  sfDatabase = "*****",
  sfSchema = "*****",
  sfWarehouse = "*****",
)

dataFrameAirlines = "/mnt/Northwoods/airlines.csv"
dfAirlines=spark.read.format("csv").option("header","true").option("inferSchema", "true").load(dataFrameAirlines)
dfAirlines.write.format("delta").mode("overwrite").save("/mnt/delta/airlines")
dfAirlines.write.format("snowflake").mode("overwrite").options(**sfOptions).option("dbtable", "DATABRICKS_AIRLINES").save()

dataFrameAirports = "/mnt/Northwoods/airports.csv"
dfAirports=spark.read.format("csv").option("header","true").option("inferSchema", "true").load(dataFrameAirports)
dfAirports.write.format("delta").mode("overwrite").save("/mnt/delta/airports")
dfAirports.write.format("snowflake").mode("overwrite").options(**sfOptions).option("dbtable", "DATABRICKS_AIRPORTS").save()

dataFrameFlights = "/mnt/Northwoods/flights.csv"
dfFlights=spark.read.format("csv").option("header","true").option("inferSchema", "true").load(dataFrameFlights)
dfFlights.write.format("delta").mode("overwrite").save("/mnt/delta/flights")
dfFlights.write.format("snowflake").mode("overwrite").options(**sfOptions).option("dbtable", "DATABRICKS_FLIGHTS").save()

In [4]:
%sql

/* 
Report: Cancellation reasons by airport 
Logic: Count of cancelled flights per cancellation reason per airport. 
*/
select 
distinct
cancellation_reason,
ap.airport,
count(flight_number) FlightCount
from delta.`/mnt/delta/flights` f 
join delta.`/mnt/delta/airports` ap on f.origin_airport=ap.iata_code
where cancelled=1
group by 
cancellation_reason,
ap.airport
order by cancellation_Reason;

cancellation_reason,airport,FlightCount
A,James M. Cox Dayton International Airport,34
A,Wilmington International Airport,4
A,Ted Stevens Anchorage International Airport,61
A,Lawton-Fort Sill Regional Airport,44
A,Lake Charles Regional Airport,8
A,Aberdeen Regional Airport,5
A,Tri-Cities Regional Airport,8
A,Gainesville Regional Airport,19
A,Manchester-Boston Regional Airport,28
A,Brainerd Lakes Regional Airport,4


In [5]:
%sql

/* 
Report: On time percentage of each airline for the year 2015 
Logic: 
On time percentage calculated for On Time Arrival as well as On Time Departure. 
Arrival Delay or Departure Delay should be less than 15 minutes as per the defintion below. 
Delay Definition: A flight is counted as "on time" if it operated less than 15 minutes later than the scheduled time shown in the carriers' Computerized Reservations Systems (CRS). Arrival performance is based on arrival at the gate. Departure performance is based on departure from the gate. (Reference: https://www.bts.gov/topics/airlines-and-airports/airline-time-performance-and-causes-flight-delays)
*/
select 
pct.airline, 
pct.iata_code,
(pct.OnTimeArrivalCount*100.0)/tot.TotalCount as OnTimeArrivalPct,
(pct.OnTimeDepartureCount*100.0)/tot.TotalCount as OnTimeDeparturePct
from
(
select
arr.airline,
arr.iata_code,
arr.OnTimeArrivalCount,
dep.OnTimeDepartureCount
from
(select 
al.airline,
f.airline as IATA_CODE,
count(flight_number)  as OnTimeArrivalCount
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airlines` al on f.airline=al.iata_code
where 
arrival_delay<15
and year=2015
group by al.airline,f.airline) arr
join
(select 
al.airline,
f.airline as IATA_CODE,
count(flight_number) as OnTimeDepartureCount
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airlines` al on f.airline=al.iata_code
where 
departure_delay<15
and year=2015
group by al.airline,f.airline) dep
on arr.iata_code=dep.iata_code) pct
join 
(select airline as IATA_CODE,count(flight_number) as TotalCount from delta.`/mnt/delta/flights` where year=2015 group by airline) tot 
on pct.IATA_CODE=tot.IATA_CODE;



airline,iata_code,OnTimeArrivalPct,OnTimeDeparturePct
United Air Lines Inc.,UA,78.15048000573952,75.41703588942126
Spirit Air Lines,NK,68.98337862820436,71.43696913417222
American Airlines Inc.,AA,80.25741614140257,81.5250749327809
Atlantic Southeast Airlines,EV,77.89963582451742,80.47473936889071
JetBlue Airways,B6,75.96649291513135,76.82214433360295
Delta Air Lines Inc.,DL,85.88518303285493,85.49254978701445
Skywest Airlines Inc.,OO,79.71727857255763,82.00757028518593
Frontier Airlines Inc.,F9,73.23087762561099,76.52142322427231
US Airways Inc.,US,79.3468032106283,83.37619203381728
American Eagle Airlines Inc.,MQ,74.07342040240027,76.02331043471177


In [6]:
%sql

/* 
Report: Delay reasons by airport 
Logic: Average Delay per Delay reason, per "Destination" airport. I found that the summation of values for delay reasons equals arrival delay, so I inferred that all delay reasons ultimately result in arrival delay, meaning delays should be associated with arrival airports. 
Arrival Delay should be >=15 as per - 
Delay Definition: A flight is counted as "on time" if it operated less than 15 minutes later than the scheduled time shown in the carriers' Computerized Reservations Systems (CRS). Arrival performance is based on arrival at the gate. Departure performance is based on departure from the gate. (Reference: https://www.bts.gov/topics/airlines-and-airports/airline-time-performance-and-causes-flight-delays)
*/

select 
ap.airport,
'Air System Delay' as DelayReason,
avg(air_system_delay) as AvgDelay
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airports` ap on f.destination_airport=ap.iata_code
where arrival_delay>=15 and air_system_delay>0
group by ap.airport,DelayReason
union
select 
ap.airport,
'Security Delay' as DelayReason,
avg(air_system_delay) as AvgDelay
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airports` ap on f.destination_airport=ap.iata_code
where arrival_delay>=15 and security_delay>0
group by ap.airport,DelayReason
union
select 
ap.airport,
'Airline Delay' as DelayReason,
avg(air_system_delay) as AvgDelay
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airports` ap on f.destination_airport=ap.iata_code
where arrival_delay>=15 and airline_delay>0
group by ap.airport,DelayReason
union
select 
ap.airport,
'Late Aircraft Delay' as DelayReason,
avg(air_system_delay) as AvgDelay
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airports` ap on f.destination_airport=ap.iata_code
where arrival_delay>=15 and late_aircraft_delay>0
group by ap.airport,DelayReason
union
select 
ap.airport,
'Weather Delay' as DelayReason,
avg(air_system_delay) as AvgDelay
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airports` ap on f.destination_airport=ap.iata_code
where arrival_delay>=15 and weather_delay>0
group by ap.airport,DelayReason


airport,DelayReason,AvgDelay
Evansville Regional Airport,Air System Delay,20.75438596491228
Newark Liberty International Airport,Air System Delay,42.20576824537003
Ralph Wien Memorial Airport,Air System Delay,12.84375
Meadows Field,Airline Delay,3.7371134020618557
Greater Binghamton Airport,Airline Delay,1.0909090909090908
Grand Forks International Airport,Airline Delay,3.9782608695652173
John Wayne Airport (Orange County Airport),Airline Delay,3.1353340883352208
Manhattan Regional Airport,Late Aircraft Delay,8.115646258503402
Ketchikan International Airport,Weather Delay,8.933333333333334
Yampa Valley Airport (Yampa Valley Regional),Air System Delay,17.333333333333332


In [7]:
%sql

/* 
Report: Airlines with the largest number of delays (Arrival)
Logic: Count number of flights with arrival delay greater than 15 minutes. 
Delay Definition: A flight is counted as "on time" if it operated less than 15 minutes later than the scheduled time shown in the carriers' Computerized Reservations Systems (CRS). Arrival performance is based on arrival at the gate. Departure performance is based on departure from the gate. (Reference: https://www.bts.gov/topics/airlines-and-airports/airline-time-performance-and-causes-flight-delays)
*/


select 
al.airline,
count(flight_number) ArrivalDelayCount
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airlines` al on f.airline=al.iata_code
where arrival_delay>=15
group by al.airline
order by count(flight_number)desc;

airline,ArrivalDelayCount
Southwest Airlines Co.,236626
American Airlines Inc.,130279
Delta Air Lines Inc.,118023
Atlantic Southeast Airlines,109184
Skywest Airlines Inc.,107795
United Air Lines Inc.,104722
American Eagle Airlines Inc.,60547
JetBlue Airways,59175
US Airways Inc.,36549
Spirit Air Lines,34221


In [8]:
%sql
/*
Report: Total number of flights by airline and airport on a monthly basis
Logic: Count of flights grouped by Month, Airline and Origin Airport. Combined YEAR and MONTH columns to convert to a date to enable Line chart visualization. Date defaulted to first day of every month. 
*/


select 
al.airline,
ap.airport as OriginAirport, 
cast(year||'-'||month||'-'||'01' as Date) as Month,
count(flight_number) FlightCount
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airlines` al on f.airline=al.iata_code
join delta.`/mnt/delta/airports` ap on f.origin_airport=ap.iata_code
group by 
al.airline,
ap.airport, 
cast(year||'-'||month||'-'||'01' as Date)


airline,OriginAirport,Month,FlightCount
Hawaiian Airlines Inc.,McCarran International Airport,2015-01-01,75
Atlantic Southeast Airlines,Brownsville/South Padre Island International Airport,2015-01-01,204
American Eagle Airlines Inc.,Will Rogers World Airport,2015-01-01,113
American Eagle Airlines Inc.,Green Bay-Austin Straubel International Airport,2015-01-01,120
US Airways Inc.,Henry E. Rohlsen Airport,2015-01-01,5
Skywest Airlines Inc.,Yuma International Airport,2015-02-01,156
American Eagle Airlines Inc.,Buffalo Niagara International Airport,2015-05-01,81
Delta Air Lines Inc.,Louis Armstrong New Orleans International Airport,2015-05-01,649
American Eagle Airlines Inc.,Dallas/Fort Worth International Airport,2015-05-01,5022
Delta Air Lines Inc.,Memphis International Airport,2015-05-01,403


In [9]:
%sql
/* 
Report: Airline with the most unique routes
Logic: 
1. Unique Routes: Minimum Flight frequency between an origin and a destination. 
2. Count unique routes per airline. 
3. Rank airlines and routes based on no. of unique routes (Most unique routes ranked higher). 
4. Sum of ranks (with all routes ranked 1) to get no. of unique routes per airline.
*/


select airline,sum(UniqueRoutesRank) as UniqueRoutesCount from
(select 
airline,origin_airport,destination_airport,
rank() over(order by FlightCount) as UniqueRoutesRank
from 
(select 
al.airline,
apo.airport as origin_airport,
apd.airport as destination_airport,
count(flight_number) FlightCount
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airlines` al on f.airline=al.iata_code
join delta.`/mnt/delta/airports` apo on f.origin_airport=apo.iata_code
join delta.`/mnt/delta/airports` apd on f.destination_airport=apd.iata_code
group by
al.airline,
apo.airport,
apd.airport))
where uniqueroutesrank=1
group by airline
order by UniqueRoutesCount desc;

airline,UniqueRoutesCount
Atlantic Southeast Airlines,68
Skywest Airlines Inc.,64
Delta Air Lines Inc.,57
United Air Lines Inc.,43
American Airlines Inc.,15
US Airways Inc.,10
American Eagle Airlines Inc.,5
JetBlue Airways,5
Southwest Airlines Co.,4
Frontier Airlines Inc.,3


In [10]:
%sql

/* 
Report: Airlines with the largest number of delays (Departure)
Logic: Count number of flights with departure delay greater than 15 minutes. 
Delay Definition: A flight is counted as "on time" if it operated less than 15 minutes later than the scheduled time shown in the carriers' Computerized Reservations Systems (CRS). Arrival performance is based on arrival at the gate. Departure performance is based on departure from the gate. (Reference: https://www.bts.gov/topics/airlines-and-airports/airline-time-performance-and-causes-flight-delays)
*/


select 
al.airline,
count(flight_number) DepartureDelayCount
from delta.`/mnt/delta/flights` f
join delta.`/mnt/delta/airlines` al on f.airline=al.iata_code
where departure_delay>=15 
group by al.airline
order by count(flight_number)desc;

airline,DepartureDelayCount
Southwest Airlines Co.,265443
American Airlines Inc.,123739
Delta Air Lines Inc.,123364
United Air Lines Inc.,120591
Atlantic Southeast Airlines,96997
Skywest Airlines Inc.,96592
JetBlue Airways,57691
American Eagle Airlines Inc.,56293
Spirit Air Lines,31602
US Airways Inc.,29144
